# Phenotype classifcation using CellX 

This notebook shows how to take segmented time lapse microscopy images and use h2b fluorescence markers to classfiy mitotic state of the cell cycle. 

The sections of this notebook are as follows:

1. Load images
2. Localise the objects
3. Classify the objects
4. Batch process

The data used in this notebook is timelapse microscopy data with h2b-gfp/rfp markers that show the spatial extent of the nucleus and it's mitotic state. 

This notebook uses the dask octopuslite image loader from the CellX/Lowe lab project.

In [1]:
from octopuslite import DaskOctopusLiteLoader
import btrack
from tqdm.auto import tqdm
import numpy as np
from scipy.special import softmax
import os
import matplotlib.pyplot as plt
from skimage.io import imread, imshow
from cellx import load_model
from cellx.tools.image import InfinitePaddedImage
from skimage.transform import resize
import glob
%matplotlib inline
plt.rcParams['figure.figsize'] = [18,8]

## 1. Load segmentation images

#### *Important:* from this point on you will need to be consistent with the use of cropping and alignment. 
Using a previously generated alignment transformation will aid greatly in the tracking notebook, which depends on the object localisation performed in this notebook. Cropping your images will ensure that no border effects from the translational shift are seen. 

In [7]:
# load images
expt = 'ND0012'
pos = 'Pos5'
root_dir = '/home/nathan/data/kraken/ras'
image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
transform_path = f'{root_dir}/{expt}/{pos}/gfp_transform_tensor.npy'
images = DaskOctopusLiteLoader(image_path, 
                               transforms=transform_path,
                               crop=(1200,1600), 
                               remove_background=False)

Using cropping: (1200, 1600)


## 2. Localise the objects
We need to also measure the mean intensity regionprops parameter in order to differentiate object class, for which we need to provide an image to measure. This means we need to provide the segmentation images twice: once to find the centroid and once to measure the pixel intensity. 

In [7]:
objects = btrack.utils.segmentation_to_objects(
    images['mask'], images['mask'],
    properties = ('area', 'max_intensity', ),
)

[INFO][2022/02/04 10:18:09 AM] Localizing objects from segmentation...
[INFO][2022/02/04 10:18:09 AM] Found intensity_image data
[INFO][2022/02/04 10:18:09 AM] Calculating weighted centroids using intensity_image
[INFO][2022/02/04 10:28:10 AM] Objects are of type: <class 'dict'>
[INFO][2022/02/04 10:28:10 AM] ...Found 9566 objects in 1738 frames.


#### Can also assign measured values from raw image to each segment using `skimage.measure.regionprops` parameters
But also need to load the raw images to be measured first. Cannot currently save out `intensity_image` parameter to object file.

In [ ]:
detailed_objects = btrack.utils.segmentation_to_objects(
    images['mask'], 
    images['gfp'],
    properties = ('area', 'mean_intensity', 'intensity_image'), 
)

In [ ]:
detailed_objects[0]

Example image showing PCNA-iRFP morphology 

In [ ]:
imshow(detailed_objects[0].properties['intensity_image'])

## 2b. Differentiate the objects based on class ID

In [ ]:
objects_gfp = [obj for obj in objects if obj.properties['max_intensity'] == 1]
objects_rfp = [obj for obj in objects if obj.properties['max_intensity'] == 2]

## 3. Classify the objects 

Load model

In [2]:
model = load_model('../models/cellx_classifier_stardist.h5')

Define normalisation functions

In [3]:
def normalize_channels(x):

    for dim in range(x.shape[-1]):
        x[..., dim] = normalize(x[..., dim])
        
    return x

def normalize(x):

    xf = x.astype(np.float32)
    mx = np.mean(xf)
    sd = np.max([np.std(xf), 1./np.prod(x.shape)])

    return (xf - mx) / sd

Define classifier function

In [12]:
def classify_objects(image,  gfp, rfp, objects, obj_type):
    
    # define stages of cell cycle to classify (dependent on model type)
    LABELS = ["interphase", "prometaphase", "metaphase", "anaphase", "apoptosis"]
    
    # iterate over frames
    for n in tqdm(range(image.shape[0])):
        
        # only select objects if in frame
        _objects = [o for o in objects if o.t == n]
        
        # empty placeholder arrays
        crops = []
        to_update = []
        
        # select h2b channel to aid in classification
        fp = gfp if obj_type == 1 else rfp
        
        # create stack by computing each frame of dask array input
        frame = np.stack(
            [image[n, ...].compute(), fp[n, ...].compute()], 
            axis=-1,) 
        
        # create padded image for network
        vol = InfinitePaddedImage(frame, mode = 'reflect')
        
        # iterate over objects 
        for obj in _objects:
            
            # create coords for image slice
            xs = slice(int(obj.x-40), int(obj.x+40), 1)
            ys = slice(int(obj.y-40), int(obj.y+40), 1)
            
            # crop image
            crop = vol[ys, xs, :]
            crop = resize(crop, (64, 64), preserve_range=True).astype(np.float32)
            
            # normalise image
            if crop.shape == (64 ,64, 2):
                crops.append(normalize_channels(crop))
                to_update.append(obj)
            else:
                print(crop.shape)
                
        if not crops:
            continue
            
        # use classifcation model to predict
        pred = model.predict(np.stack(crops, axis=0))
        
        # check correct number of predictions
        assert pred.shape[0] == len(_objects)
        
        # assign labels to objects
        for idx in range(pred.shape[0]):
            obj = _objects[idx]
            
            # assigning details of prediction
            pred_label = np.argmax(pred[idx, ...])
            pred_softmax = softmax(pred[idx, ...])
            logits = {f"prob_{k}": pred_softmax[ki] for ki, k in enumerate(LABELS)}
            
            # write out
            obj.label = pred_label
            obj.properties = logits

    return objects

#### Load raw images for classifier, a colour channel dependent on `obj_type` needed too (i.e. GFP is `obj_type = 1`, RFP is `obj_type = 2`

In [ ]:
bf = images['brightfield']
gfp = images['gfp']
rfp = images['rfp']

#### Classify objects

In [ ]:
objects_gfp = classify_objects(bf, objects_gfp, obj_type = 1)
objects_rfp = classify_objects(bf, objects_rfp, obj_type = 2)

#### Inspect an example object

In [ ]:
objects_gfp[0]

#### Save out classified GFP objects

In [ ]:
with btrack.dataio.HDF5FileHandler(
    f'{root_dir}/{expt}/{pos}/objects_type_1.h5', 'w', obj_type='obj_type_1',
) as hdf:
    #hdf.write_segmentation(masks['mask'])
    hdf.write_objects(objects_gfp)

#### Save out classified RFP objects

In [ ]:
with btrack.dataio.HDF5FileHandler(
    f'{root_dir}/{expt}/{pos}/objects_type_2.h5', 'w', obj_type='obj_type_2',
) as hdf:
    #hdf.write_segmentation(masks['mask'])
    hdf.write_objects(objects_rfp)

# 4. Batch process
Iterate over many experiments and positions (need to ensure you define normalisation and classification functions above first)

In [ ]:
%%timeit
root_dir = '/home/nathan/data/kraken/ras'
expt_list = ['ND0010', 'ND0011', 'ND0012', 'ND0013']
pos_list = 'all'
overwrite = True

for expt in tqdm(expt_list):
    
    # Find all positions in that experiment, if pos_list is all then it finds all positions
    if pos_list == 'all':
        pos_list = [pos for pos in os.listdir(f'{root_dir}/{expt}') 
                if 'Pos' in pos 
                and os.path.isdir(f'{root_dir}/{expt}/{pos}')]  

    ### Iterate over all positions in that experiment
    for pos in tqdm(pos_list):

        ### check if overwrite param is false check if raw directory already created and if type of transform file already exists and decide whether to skip pos
        if not overwrite and glob.glob(f'{root_dir}/{expt}/{pos}/*objects*.h5'):
            print(glob.glob(f'{root_dir}/{expt}/{pos}/*objects*.h5'), f'file found, skipping {expt}/{pos}')
            continue

        print(f'Starting {expt}/{pos}')
        # load segmentation images and apply necessary transforms and crops
        image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
        transform_path = f'{root_dir}/{expt}/{pos}/gfp_transform_tensor.npy'
        images = DaskOctopusLiteLoader(image_path, 
                           transforms=transform_path,
                           crop=(1200,1600), 
                           remove_background=False)

        # ID the objects in each segmentation image and assign option properties to them
        objects = btrack.utils.segmentation_to_objects(
            images['mask'], 
            properties = ('area',),
            assign_class_ID = True
        )

        # differentiate the objects based on class ID
        objects_gfp = [obj for obj in objects if obj.properties['class id'] == 1]
        objects_rfp = [obj for obj in objects if obj.properties['class id'] == 2]

        # load classifcation model and define labels
        model = load_model('../models/cellx_classifier_stardist.h5')
        LABELS = ["interphase", "prometaphase", "metaphase", "anaphase", "apoptosis"]
        
        # load images for classifcation
        bf = images['brightfield']
        gfp = images['gfp']
        rfp = images['rfp']

        # classify objects
        print("Classifying objects")
        objects_gfp = classify_objects(bf, gfp, rfp, objects_gfp, obj_type = 1)
        objects_rfp = classify_objects(bf, gfp, rfp, objects_rfp, obj_type = 2)

        # save out classified objects as segmentation h5 file
        with btrack.dataio.HDF5FileHandler(
            f'{root_dir}/{expt}/{pos}/objects_type_1.h5', 'w', obj_type='obj_type_1',
        ) as hdf:
            #hdf.write_segmentation(masks['mask'])
            hdf.write_objects(objects_gfp)
        with btrack.dataio.HDF5FileHandler(
            f'{root_dir}/{expt}/{pos}/objects_type_2.h5', 'w', obj_type='obj_type_2',
        ) as hdf:
            #hdf.write_segmentation(masks['mask'])
            hdf.write_objects(objects_rfp)     

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

Starting ND0010/Pos5
Using cropping: (1200, 1600)


[INFO][2022/02/07 09:38:15 PM] Localizing objects from segmentation...
[INFO][2022/02/07 09:41:18 PM] Objects are of type: <class 'dict'>
[INFO][2022/02/07 09:41:19 PM] ...Found 363732 objects in 1193 frames.


Classifying objects


...

[INFO][2022/02/08 11:07:07 AM] Opening HDF file: /home/nathan/data/kraken/ras/ND0010/Pos0/objects_type_1.h5...
[INFO][2022/02/08 11:07:24 AM] Writing objects/obj_type_1
[INFO][2022/02/08 11:07:24 AM] Writing labels/obj_type_1
[INFO][2022/02/08 11:07:24 AM] Loading objects/obj_type_1 (2346248, 5) (2346248 filtered: None)
[INFO][2022/02/08 11:07:51 AM] Writing properties/obj_type_1/area (2346248,)
[INFO][2022/02/08 11:07:51 AM] Writing properties/obj_type_1/class id (2346248,)
[INFO][2022/02/08 11:07:51 AM] Writing properties/obj_type_1/prob_interphase (2346248,)
[INFO][2022/02/08 11:07:52 AM] Writing properties/obj_type_1/prob_prometaphase (2346248,)
[INFO][2022/02/08 11:07:52 AM] Writing properties/obj_type_1/prob_metaphase (2346248,)
[INFO][2022/02/08 11:07:52 AM] Writing properties/obj_type_1/prob_anaphase (2346248,)
[INFO][2022/02/08 11:07:52 AM] Writing properties/obj_type_1/prob_apoptosis (2346248,)
[INFO][2022/02/08 11:07:53 AM] Closing HDF file: /home/nathan/data/kraken/ras/ND00

Starting ND0010/Pos2
Using cropping: (1200, 1600)


[INFO][2022/02/08 11:07:59 AM] Localizing objects from segmentation...
[INFO][2022/02/08 11:14:17 AM] Objects are of type: <class 'dict'>
[INFO][2022/02/08 11:14:37 AM] ...Found 3070256 objects in 1189 frames.


Classifying objects


  0%|          | 0/1189 [00:00<?, ?it/s]

# Parallel batch process

In [7]:
def classify(pos):
    ### check if overwrite param is false check if raw directory already created and if type of transform file already exists and decide whether to skip pos
    if not overwrite and glob.glob(f'{root_dir}/{expt}/{pos}/*objects*.h5'):
        print(glob.glob(f'{root_dir}/{expt}/{pos}/*objects*.h5'), f'file found, skipping {expt}/{pos}')
        return

    print(f'Starting {expt}/{pos}')
    # load segmentation images and apply necessary transforms and crops
    image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
    transform_path = f'{root_dir}/{expt}/{pos}/gfp_transform_tensor.npy'
    images = DaskOctopusLiteLoader(image_path, 
                       transforms=transform_path,
                       crop=(1200,1600), 
                       remove_background=False)

    # ID the objects in each segmentation image and assign option properties to them
    objects = btrack.utils.segmentation_to_objects(
        images['mask'], images['mask'],
        properties = ('area', 'max_intensity', ),
    )

    # differentiate the objects based on class ID
    objects_gfp = [obj for obj in objects if obj.properties['max_intensity'] == 1]
    objects_rfp = [obj for obj in objects if obj.properties['max_intensity'] == 2]

    # load classifcation model and define labels
    model = load_model('../models/cellx_classifier_stardist.h5')
    LABELS = ["interphase", "prometaphase", "metaphase", "anaphase", "apoptosis"]

    # load images for classifcation
    bf = images['brightfield']
    gfp = images['gfp']
    rfp = images['rfp']

    # classify objects
    print("Classifying objects")
    objects_gfp = classify_objects(bf, objects_gfp, obj_type = 1)
    objects_rfp = classify_objects(bf, objects_rfp, obj_type = 2)

    # save out classified objects as segmentation h5 file
    with btrack.dataio.HDF5FileHandler(
        f'{root_dir}/{expt}/{pos}/objects_type_1.h5', 'w', obj_type='obj_type_1',
    ) as hdf:
        #hdf.write_segmentation(masks['mask'])
        hdf.write_objects(objects_gfp)
    with btrack.dataio.HDF5FileHandler(
        f'{root_dir}/{expt}/{pos}/objects_type_2.h5', 'w', obj_type='obj_type_2',
    ) as hdf:
        #hdf.write_segmentation(masks['mask'])
        hdf.write_objects(objects_rfp)     

    return

In [8]:
from multiprocessing import Pool
cpus = os.cpu_count()
cpus

12

In [11]:
pos_list = [pos for pos in os.listdir(f'{root_dir}/{expt}') 
                    if 'Pos' in pos 
                    and os.path.isdir(f'{root_dir}/{expt}/{pos}')]
pos_list

['Pos5',
 'Pos11',
 'Pos3',
 'Pos1',
 'Pos8',
 'Pos10',
 'Pos0',
 'Pos2',
 'Pos6',
 'Pos7',
 'Pos9',
 'Pos4']

In [13]:
for expt in expt_list:
    if __name__ == '__main__':
        with Pool(cpus) as p:
            p.map(classify, pos_list)

Starting ND0010/Pos3Starting ND0010/Pos4Starting ND0010/Pos11Starting ND0010/Pos5Starting ND0010/Pos7Starting ND0010/Pos1Starting ND0010/Pos10Starting ND0010/Pos0Starting ND0010/Pos8
Starting ND0010/Pos2
Starting ND0010/Pos9

Starting ND0010/Pos6


Using cropping: (1200, 1600)Using cropping: (1200, 1600)

Using cropping: (1200, 1600)Using cropping: (1200, 1600)Using cropping: (1200, 1600)Using cropping: (1200, 1600)
Using cropping: (1200, 1600)

Using cropping: (1200, 1600)




Using cropping: (1200, 1600)
Using cropping: (1200, 1600)

Using cropping: (1200, 1600)

Using cropping: (1200, 1600)



[INFO][2022/02/07 05:18:20 PM] Localizing objects from segmentation...
[INFO][2022/02/07 05:18:20 PM] Found intensity_image data
[INFO][2022/02/07 05:18:20 PM] Localizing objects from segmentation...
[INFO][2022/02/07 05:18:20 PM] Calculating weighted centroids using intensity_image
[INFO][2022/02/07 05:18:20 PM] Localizing objects from segmentation...
[INFO][2022/02/07 05:18:20 PM] Found intensity_image data
[INFO][2022/02/07 05:18:20 PM] Calculating weighted centroids using intensity_image
[INFO][2022/02/07 05:18:20 PM] Localizing objects from segmentation...
[INFO][2022/02/07 05:18:20 PM] Found intensity_image data
[INFO][2022/02/07 05:18:20 PM] Found intensity_image data
[INFO][2022/02/07 05:18:20 PM] Calculating weighted centroids using intensity_image
[INFO][2022/02/07 05:18:20 PM] Localizing objects from segmentation...
[INFO][2022/02/07 05:18:20 PM] Calculating weighted centroids using intensity_image
[INFO][2022/02/07 05:18:20 PM] Localizing objects from segmentation...
[INFO]

KeyboardInterrupt: 